In [1]:
import os
import pandas as pd
# create the feature- and actuals-data list
# set the feature and actuals year lists
feature_years = ['2017','2018','2019','2020']
actual_years = ['2018','2019','2020','2021']

actuals_df_list = []
features_df_list = []

# path to the current directory
current_dir = os.getcwd()

for i in range(len(feature_years)):
    # relative paths to the parquet files
    relative_path_features = os.path.join('..', 'data', 'cm_features_to_oct' + feature_years[i] + '.parquet')
    relative_path_actuals = os.path.join('..', 'data', 'cm_actuals_' + actual_years[i] + '.parquet')

    path_features = os.path.join(current_dir, relative_path_features)
    path_actuals = os.path.join(current_dir, relative_path_actuals)

    # append datasets to the lists
    actuals_df_list.append({'year':actual_years[i], 'data':pd.read_parquet(path_actuals, engine='pyarrow')})
    features_df_list.append({'year':feature_years[i], 'data':pd.read_parquet(path_features, engine='pyarrow')})

# concat the feature datasets, so that every data contains the observations starting with january 1990
for i in range(1,len(features_df_list)):
    features_df_list[i]['data'] = pd.concat([features_df_list[i-1]['data'], features_df_list[i]['data']])

country_list = sorted(features_df_list[3]['data'].index.get_level_values('country_id').unique().tolist())

# country group list of all four datasets
country_feature_group_list = []
country_actual_group_list = []
# fill list 
for i in range(len(features_df_list)):
    country_feature_group_list.append(features_df_list[i]['data'].groupby('country_id'))
    country_actual_group_list.append(actuals_df_list[i]['data'].groupby('country_id'))

In [2]:
import pandas as pd
from joblib import load
import numpy as np
import os

# Pfad zur Parquet-Datei
parquet_file_path = r'C:\Users\Tobias\Documents\BAconflictPrediction\ConflictPrediction\benchmark views\bm_cm_last_historical_poisson_expanded_2018.parquet'

# Parquet-Datei in ein DataFrame einlesen
benchmark_data = pd.read_parquet(parquet_file_path)

benchmark_data

outcome
month_id country_id draw         
457      1          0           0
                    1           0
                    2           0
                    3           0
                    4           0
...                           ...
468      246        995        22
                    996        19
                    997        21
                    998        14
                    999        10

[2292000 rows x 1 columns]

In [3]:
user_dir = os.path.expanduser('~')
file_path = os.path.join(user_dir, 'iCloudDrive\\Joblib BA\\final baseline', 'FinalTask2_baseline_predct_hurdleWmax24.joblib')
loaded_vars_baseline = load(file_path)

task2_baseline_list = loaded_vars_baseline[0] # crps averages for all 4 datasets
w_minimization_list = loaded_vars_baseline[1] # contains the minimal w's for the different baselines for each year and country
baseline_prediction_list = loaded_vars_baseline[2] # predictions with the minimal w's for each dataset and country
baseline1_average_crps = loaded_vars_baseline[3] # mean CRPS from the baseline_prediction_list
baseline2_average_crps = loaded_vars_baseline[4] # ""
baseline3_average_crps = loaded_vars_baseline[5]
baseline4_average_crps = loaded_vars_baseline[6]

print('Overall CRPS, max. w = 24')
print('baseline 1: ' + str(np.round(baseline1_average_crps, decimals = 4)))
print('baseline 2: ' + str(np.round(baseline2_average_crps, decimals = 4)))
print('baseline 3: ' + str(np.round(baseline3_average_crps, decimals = 4)))
print('baseline 4: ' + str(np.round(baseline4_average_crps, decimals = 4)))

Overall CRPS, max. w = 24
baseline 1: 16.1644
baseline 2: 16.3591
baseline 3: 16.2584
baseline 4: 16.2204


In [4]:
prediction_year = '2018' # 2019, 2020, 2021

dataset_index = actual_years.index(prediction_year)
actual_data = actuals_df_list[dataset_index]['data']
actuals_months = actual_data.index.get_level_values('month_id').unique()

In [5]:
monthly_fatlities_pred = pd.DataFrame()

#### achtung nbinom 1, hurdle 0
for country in baseline_prediction_list[0]:

    for s in country['prediction'][prediction_year][0]['s']:
        
        month_id = actuals_months[s-3]
        country_id = country['country_id']

        distribution = country['prediction'][prediction_year][0]['fatalities']
        distribution = [int(num) for num in distribution]
        draw = list(range(0,len(distribution)))

        index_tupel_list = []
        for i in range(len(distribution)):
            index_tupel_list.append((month_id,country_id,draw[i]))

        index = pd.MultiIndex.from_tuples(index_tupel_list, names=['month_id', 'country_id', 'draw'])

        # Erstelle eine Beispiel-Spalte "outcome"
        outcome = distribution

        # Erstelle den DataFrame
        df = pd.DataFrame({'outcome': outcome}, index=index)

        monthly_fatlities_pred = pd.concat([monthly_fatlities_pred, df], ignore_index=False)

In [6]:
import CRPS.CRPS as pscore
import numpy as np

actual_group = actual_data.groupby('country_id')

baseline_country_group = monthly_fatlities_pred.groupby('country_id')
crps_values_test = []

for country in baseline_country_group:
    country_id = country[0]

    country_crps_list = []
    for i in range(0,12):
        y_true = actual_group.get_group(country_id).iloc[i,0]
        month = actuals_months[i]
        prediction = baseline_country_group.get_group(country_id).xs(month, level='month_id').values.flatten()
        crps = pscore(prediction,y_true).compute()[0]
        country_crps_list.append(crps)
    crps_values_test.append(np.mean(country_crps_list))

mean_crps_test = np.mean(crps_values_test)


print('Mean CRPS = ' + str(mean_crps_test) + ' \\'+'\\')

Mean CRPS = 14.359802036155083 \\


In [7]:
monthly_fatlities_pred.to_parquet('cm_hurdle_test_window_2018.parquet')


In [8]:
monthly_fatlities_pred

outcome
month_id country_id draw         
457      1          0           0
                    1           0
                    2           0
                    3           0
                    4           0
...                           ...
468      246        994       478
                    995       503
                    996       536
                    997       582
                    998       662

[2289708 rows x 1 columns]